In [17]:
#!pip3 install langchain

## Import packages

In [26]:
from dotenv import load_dotenv
# from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import AzureOpenAIEmbeddings

from langchain.vectorstores import ElasticsearchStore
from langchain.text_splitter import CharacterTextSplitter
from urllib.request import urlopen
import os, json

load_dotenv()
 
# openai_api_key=os.getenv('OPENAI_API_KEY')
elastic_user=os.getenv('ES_USER')
elastic_password=os.getenv('ES_PASSWORD')
elastic_endpoint=os.getenv("ES_ENDPOINT")
elastic_index_name='elasticsearch-store'

# Add documents

## Let's read the sample dataset and deserialize the document.

In [27]:
with open('workplace-docs.json') as f:
   workplace_docs = json.load(f)
 
print(f"Successfully loaded {len(workplace_docs)} documents")

Successfully loaded 15 documents


In [28]:
metadata = []
content = []

for doc in workplace_docs:
  content.append(doc["content"])
  metadata.append({
      "name": doc["name"],
      "summary": doc["summary"],
      "rolePermissions":doc["rolePermissions"]
  })

text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0)
docs = text_splitter.create_documents(content, metadatas=metadata)

Created a chunk of size 245, which is longer than the specified 50
Created a chunk of size 288, which is longer than the specified 50
Created a chunk of size 204, which is longer than the specified 50
Created a chunk of size 281, which is longer than the specified 50
Created a chunk of size 249, which is longer than the specified 50
Created a chunk of size 285, which is longer than the specified 50
Created a chunk of size 298, which is longer than the specified 50
Created a chunk of size 270, which is longer than the specified 50
Created a chunk of size 224, which is longer than the specified 50
Created a chunk of size 288, which is longer than the specified 50
Created a chunk of size 260, which is longer than the specified 50
Created a chunk of size 199, which is longer than the specified 50
Created a chunk of size 290, which is longer than the specified 50
Created a chunk of size 251, which is longer than the specified 50
Created a chunk of size 195, which is longer than the specifie

In [29]:
docs

[Document(page_content='Effective: March 2020\nPurpose', metadata={'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy reviews and updates. Employees are encouraged to direct any questions or concerns', 'rolePermissions': ['demo', 'manager']}),
 Document(page_content='The purpose of this full-time work-from-home policy is to provide guidelines and support for employees to conduct their work remotely, ensuring the continuity and productivity of business operations during the COVID-19 pandemic and beyond.\nScope', metadata={'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication

In [30]:
from elasticsearch import Elasticsearch

# embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

model_name = os.getenv('MODEL_NAME')
azure_endpoint = os.getenv('AZURE_ENDPOINT')
azure_api_key = os.getenv('AZURE_API_KEY')
azure_openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')

embeddings = AzureOpenAIEmbeddings(
    model=model_name,
    azure_endpoint=azure_endpoint, 
    api_key=azure_api_key,
    openai_api_version=azure_openai_api_version
)
 
url = f"https://{elastic_user}:{elastic_password}@{elastic_endpoint}:9200"
connection = Elasticsearch(url, ca_certs = "./http_ca.crt", verify_certs = True)

 
es = ElasticsearchStore.from_documents( 
                            docs,
                            embedding = embeddings, 
                            es_url = url, 
                            es_connection = connection,
                            index_name = elastic_index_name, 
                            es_user = elastic_user,
                            es_password = elastic_password)

In [31]:
es

In [32]:
def showResults(output):
  print("Total results: ", len(output))
  for index in range(len(output)):
    print(output[index])

# Similarity / Vector Search (Approximate KNN Search) - ApproxRetrievalStrategy()

In [33]:
query = "work from home policy"
result = es.similarity_search(query=query)

showResults(result)

Total results:  4
page_content='The purpose of this full-time work-from-home policy is to provide guidelines and support for employees to conduct their work remotely, ensuring the continuity and productivity of business operations during the COVID-19 pandemic and beyond.\nScope' metadata={'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy reviews and updates. Employees are encouraged to direct any questions or concerns', 'rolePermissions': ['demo', 'manager']}
page_content='This work-from-home policy will be reviewed periodically and updated as necessary, taking into account changes in public health guidance, business needs, and employee feedback.\nQuestions and Concerns' metadata={'name': 'Work From Home Polic

# Hybrid Search (Approximate KNN + Keyword Search) - ApproxRetrievalStrategy()

In [34]:
# embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


es = ElasticsearchStore(
    es_url = url,
    es_connection = connection,
    es_user=elastic_user,
    es_password=elastic_password,
    embedding=embeddings,
    index_name=elastic_index_name,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(
        hybrid=True
    )
)

es.similarity_search("work from home policy")

[Document(page_content='This work-from-home policy will be reviewed periodically and updated as necessary, taking into account changes in public health guidance, business needs, and employee feedback.\nQuestions and Concerns', metadata={'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy reviews and updates. Employees are encouraged to direct any questions or concerns', 'rolePermissions': ['demo', 'manager']}),
 Document(page_content='The purpose of this full-time work-from-home policy is to provide guidelines and support for employees to conduct their work remotely, ensuring the continuity and productivity of business operations during the COVID-19 pandemic and beyond.\nScope', metadata={'name': 'Work From Home

# Exact KNN Search (Brute Force) - ExactRetrievalStrategy()

In [35]:
# embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

es = ElasticsearchStore(
    es_url = url,
    es_connection = connection,
    es_user=elastic_user,
    es_password=elastic_password,
    embedding=embeddings,
    index_name=elastic_index_name,
    strategy=ElasticsearchStore.ExactRetrievalStrategy()
)

es.similarity_search("work from home policy")

[Document(page_content='The purpose of this full-time work-from-home policy is to provide guidelines and support for employees to conduct their work remotely, ensuring the continuity and productivity of business operations during the COVID-19 pandemic and beyond.\nScope', metadata={'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy reviews and updates. Employees are encouraged to direct any questions or concerns', 'rolePermissions': ['demo', 'manager']}),
 Document(page_content='This work-from-home policy will be reviewed periodically and updated as necessary, taking into account changes in public health guidance, business needs, and employee feedback.\nQuestions and Concerns', metadata={'name': 'Work From Home

In [39]:
# embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# In the following, we choose .elser_model_2 instead of .elser_model_1

es = ElasticsearchStore.from_documents(
    docs,
    es_url = url,
    es_connection = connection,
    es_user=elastic_user,
    es_password=elastic_password,
    index_name=elastic_index_name+"-"+"elser",
    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(model_id=".elser_model_2")
)


# Index / Search Documents using ELSER - SparseVectorRetrievalStrategy()

In [40]:
es

In [41]:
es.similarity_search("work from home policy")

/usr/local/lib/python3.9/site-packages/langchain_community/vectorstores/elasticsearch.py:828: ElasticsearchWarning: text_expansion is deprecated. Use sparse_vector instead.
  response = self.client.search(


[Document(page_content='This policy applies to all employees who are eligible for remote work as determined by their role and responsibilities. It is designed to allow employees to work from home full time while maintaining the same level of performance and collaboration as they would in the office.\nEligibility', metadata={'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy reviews and updates. Employees are encouraged to direct any questions or concerns', 'rolePermissions': ['demo', 'manager'], 'name': 'Work From Home Policy'}),
 Document(page_content='This policy applies to all employees who are eligible for remote work as determined by their role and responsibilities. It is designed to allow employees to work from home full time while maint